# Exploring the Reddit PRAW API

In [17]:
# Libraries

import requests
import praw
import configparser
import pandas as pd
import numpy as np
import nltk
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/Users/kanithamann/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# The following cells uses an INI file to pull in credentials needed to access the PRAW API.
# This INI file is stored locally only

config = configparser.RawConfigParser()
config.read("config.txt")
reddit = praw.Reddit(client_id=config.get("reddit","client_id"),
                     client_secret=config.get("reddit","client_secret"),
                     password=config.get("reddit","password"),
                     user_agent="Political exploration",
                     username=config.get("reddit","username"))


## General Data

The API can look at a subreddit and pull information like the title, url, and body of a post (if it's not a link post).
We can also get the karma score, the number of comments, and when the post was created.

In [18]:
# Example data pull

posts = []

for post in reddit.subreddit('politics').hot(limit=10):
    posts.append([post.title, 
                  post.score, 
                  post.id, 
                  post.subreddit, 
                  post.url, 
                  post.num_comments, 
                  post.selftext, 
                  datetime.utcfromtimestamp(post.created)
                 ])
df = pd.DataFrame(posts,
                     columns=['title', 
                              'score', 
                              'id', 
                              'subreddit', 
                              'url', 
                              'num_comments', 
                              'body', 
                              'created'
                             ])

df

,title,score,id,subreddit,url,num_comments,body,created
0,We are the majority-makers in Congress. Meet t...,1394,bugtra,politics,https://www.reddit.com/r/politics/comments/bug...,1177,The New Democrat Coalition is a group of 101 f...,2019-05-30 00:30:20
1,Conservatives Stunned by Mueller Suggesting Tr...,25462,butf0t,politics,https://nymag.com/intelligencer/2019/05/conser...,2737,,2019-05-30 21:51:12
2,Trump says Russia 'helped me get elected' for ...,63781,buscyw,politics,https://www.independent.co.uk/news/world/ameri...,6690,,2019-05-30 20:08:47
3,Trump Finally Admits Russia Helped Him Get Ele...,9102,busa98,politics,https://www.thedailybeast.com/trump-finally-ad...,430,,2019-05-30 20:00:59
4,Sen. Elizabeth Warren on The View: If Trump we...,3353,buuqoh,politics,https://abcnews.go.com/Politics/sen-elizabeth-...,268,,2019-05-30 23:46:59
5,The USS John McCain Debacle Shows the Whole Co...,4718,butiti,politics,https://www.esquire.com/news-politics/politics...,333,,2019-05-30 22:00:38
6,Fox News's Shep Smith: Mueller statement 'dire...,9584,burvdr,politics,https://thehill.com/homenews/media/446033-fox-...,403,,2019-05-30 19:12:51
7,Trump Attacks Mueller Probe - Inadvertently Co...,6820,busfjn,politics,https://www.haaretz.com/us-news/trump-attacks-...,312,,2019-05-30 20:16:01
8,Republican group to hand-deliver Mueller repor...,3277,buufm0,politics,https://thehill.com/homenews/house/446140-repu...,171,,2019-05-30 23:20:04
9,Sun-Maid Pulls Baseball Sponsorship After Team...,7207,bus1mi,politics,https://www.newsweek.com/sun-maid-alexandria-o...,778,,2019-05-30 19:33:01


## Narrowing down the data

### By Subreddit

Michael Bennet, Steve Bullock, Julián Castro, Bill de Blasio, John Delaney, John Hickenlooper, Seth Moulton, Tim Ryan, Eric Swalwell, Marianne Williamson, and Andrew Yang are candidates who do not have dedicated subreddits as of May 30, 2019.

For the other candidates, here are their subscriber counts as of May 30, 2019

In [8]:
subreddits = {'Joe Biden': 'JoeBiden',
              'Cory Booker': 'corybooker',
              'Pete Buttigieg': 'Pete_Buttigieg',
              'Tulsi Gabbard': 'tulsi',
              'Kirsten Gillibrand': 'Kirsten_Gillibrand',
              'Mike Gravel': 'gravelforpresident',
              'Kamala Harris': 'Kamala',
              'Jay Inslee': 'inslee2020',
              'Amy Klobuchar': 'BaemyKlobaechar',
              'Beto O\'Rourke': 'Beto2020',
              'Bernie Sanders': 'SandersForPresident',
              'Donald Trump': 'The_Donald',
              'Elizabeth Warren': 'ElizabethWarren',
              'politics': 'politics'}

for subreddit in subreddits.values():
    print(subreddit,': ',reddit.subreddit(subreddit).subscribers)

JoeBiden :  952
corybooker :  378
Pete_Buttigieg :  22156
tulsi :  10607
Kirsten_Gillibrand :  106
gravelforpresident :  3660
Kamala :  1617
inslee2020 :  546
BaemyKlobaechar :  502
Beto2020 :  10131
SandersForPresident :  249726
The_Donald :  747188
ElizabethWarren :  11381
politics :  5120363


## By Date

To narrow down by date, we can either use the `reddit.subreddit('subreddit').search` functionality, or make the call with conditions set around the `post.created` parameter.

## Feature Engineering

Possible ideas include:

1. Number of average and cumulative posts in a subreddit, but the API doesn't make this easy to get.
2. The karma/score and number of comments for an associated headline.
3. Topic modeling on the headline
4. What candidate is addressed in the headline
5. Sentiment analysis on the headline
6. Counting if the word "donation" is referenced in the headline (better yet, the comments)

In [14]:
# Using id = buf84a as a test

sample_post = reddit.submission(id='buf84a')

sample_post.title

'Megathread: Robert Mueller to Make Public Statement About Russia Investigation'

In [ ]:
# This runs too long

sample_post.comments.replace_more(limit=None)

i = 0

for top_level_comment in sample_post.comments:
    if 'donation' in top_level_comment.body:
        i+=1
    #print(top_level_comment.body)
    
print(i)

## Sentiment Analysis

In [29]:
sia = SIA()
results = []

for i in range(0,len(posts)):
    line = posts[i][0]
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    #results.append(pol_score)
    print(pol_score)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'We are the majority-makers in Congress. Meet the New Dems! A(us)A'}
{'neg': 0.13, 'neu': 0.648, 'pos': 0.222, 'compound': 0.25, 'headline': 'Conservatives Stunned by Mueller Suggesting Trump Is Not Innocent'}
{'neg': 0.209, 'neu': 0.791, 'pos': 0.0, 'compound': -0.5719, 'headline': "Trump says Russia 'helped me get elected' for first time in furious outburst at Mueller"}
{'neg': 0.0, 'neu': 0.761, 'pos': 0.239, 'compound': 0.296, 'headline': 'Trump Finally Admits Russia Helped Him Get Elected'}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': "Sen. Elizabeth Warren on The View: If Trump weren't president 'he'd be in handcuffs'"}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'The USS John McCain Debacle Shows the Whole Country Has to Cater to President* Snowflake'}
{'neg': 0.204, 'neu': 0.796, 'pos': 0.0, 'compound': -0.3182, 'headline': "Fox News's Shep Smith: Mueller statement 'dire

In [ ]:
# Save to a csv file if needed for the future

posts.to_csv('reddit_posts.csv', index=False) 